# Generating flow duration and flow size
**NOTE** In this version I am now expanding the neural network to generate a third value: Idle time. This will be used as IAT time kinda to use within the simulation.

In [8]:
# Import dependencies
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm.notebook import tqdm

## Constants

In [ ]:
TRAINING_DATA_LENGTH = 924689
BATCH_SIZE = 300
NUM_EPOCHS = 40
LEARNING_RATE = 0.0001
RANDOM_SEED = 77
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TRAINING_DATA = "Training Data\data.pt"
PATH = "generator1.pt"

## Discriminator class

In [9]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            # Input is 2D, first hidden layer is composed of 256 neurons with ReLU activation
            nn.Linear(3, 128), 
            nn.ReLU(),

            # Have to use dropout to avoid overfitting
            nn.Dropout(0.3),

            # second and third layers are composed to 128 and 64 neurons, respectively
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            # output is composed of a single neuron with sigmoidal activation to represent a probability
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Generator class 

In [10]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
        )

    def forward(self, x):
        output = self.model(x)
        return output

## Weights and biases function

In [ ]:
def weights_biases_init(model):
    classname = model.__class__.__name__
    if classname.find("Linear") != -1:
        nn.init.kaiming_normal_(model.weight, mode='fan_in', nonlinearity='relu')
        nn.init.zeros_(model.bias)

## Prepare data

In [11]:
def train_data_length(data, length):
    return data[:length]
    
def load_data():
    data = torch.load("data.pt")
    data = data.to(torch.float32)
    train_data = train_data_length(data,TRAINING_DATA_LENGTH)
    return train_data

In [12]:
train_data = load_data()
train_data = train_data.to(DEVICE)
train_labels = torch.zeros(TRAINING_DATA_LENGTH)
train_labels = train_labels.to(DEVICE)
train_set = [(train_data[i], train_labels[i]) for i in range(TRAINING_DATA_LENGTH)]
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, drop_last = True)

print("Done")

Done


In [ ]:
def convert_time(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d hours %02d mins %02d secs" % (hour, minutes, seconds)

## Prepare for training

In [13]:
generator = Generator()
discriminator = Discriminator()

# Before training the models, need to define some parameters
loss_function = nn.BCELoss()
optimiser_discriminator = optim.Adam(discriminator.parameters(), lr = LEARNING_RATE)
optimiser_generator = optim.Adam(generator.parameters(), lr = LEARNING_RATE)

# Create empty loss lists to track values and visualise
generator_loss_values = []
discriminator_loss_values = []
epoch_count = []

## Training process

In [14]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):

    # Taking the real samples of the current batch from the data loader and assign them to real_samples
    # The first dimension of the tensor has the number of elements equal to the batch size. 
    # This is the standard way of organising data in PyTorch, with each line of the tensor representing one sample from the batch.
    for n, (real_samples, _) in enumerate(tqdm(train_loader)):
        # DATA FOR TRAINING THE DISCRIMINATOR

        # Using torch.ones() to create labels with the value 1 for real samples, and then assign to real_samples_labels
        real_samples_labels = torch.ones((BATCH_SIZE, 1))

        # Create the generated samples by storing random data in latent_space_samples
        # This is fed into the generator to obtain generated_samples
        torch.manual_seed(RANDOM_SEED)
        latent_space_samples = torch.randn((BATCH_SIZE, 3))
        generated_samples = generator(latent_space_samples)


        # Use torch.zeros() to assign 0 to the labels for the generated samples
        generated_samples_labels = torch.zeros((BATCH_SIZE, 1))


        # Concatenate the real and generated samples and labels and store them in all_samples
        # and all_samples_labels to train the discriminator
        all_samples = torch.cat((real_samples, generated_samples))
        all_samples_labels = torch.cat((real_samples_labels, generated_samples_labels))



        # TRAINING THE DISCRIMINATOR

        # Clear the gradients at each training step to avoid accumulating them
        discriminator.zero_grad()

        # Calculate the output of the discriminator from the training data in all_samples
        output_discriminator = discriminator(all_samples)

        # Calculate the loss function using discriminator output and all the labels
        loss_discriminator = loss_function(output_discriminator, all_samples_labels)
        
        # Calculate the gradients to update the weights
        loss_discriminator.backward()

        # Update the discriminator weights
        optimiser_discriminator.step()



        # DATA FOR TRAINING THE GENERATOR
        
        # Storing random data in latent_space_samples with a number of lines to equal batch_size
        torch.manual_seed(RANDOM_SEED)
        latent_space_samples = torch.randn((BATCH_SIZE, 3))

        # TRAINING THE GENERATOR
        generator.zero_grad()
        generated_samples = generator(latent_space_samples)

        # Feeding generator's output into the discriminator and store its output, which is used
        # as the output of the whole model
        output_discriminator_generated = discriminator(generated_samples)

        # Calculate the loss function
        loss_generator = loss_function(output_discriminator_generated, real_samples_labels)
        
        # Calculate and update the gradients
        # REMEMBER:
        # When the generator is trained, the discriminator weights are frozen since optimiser_generator
        # was created with its first argument equal to generator.parameters()
        loss_generator.backward()
        optimiser_generator.step()


        # Show loss
        if epoch % 10 == 0 and n == BATCH_SIZE - 1:
            epoch_count.append(epoch)
            generator_loss_values.append(generator_loss.cpu().detach().numpy())
            discriminator_loss_values.append(discriminator_loss.cpu().detach().numpy())
            print(f"Epoch: {epoch} | D Loss: {discriminator_loss} | G Loss: {generator_loss}")

end_time = time.time()
run_time = round(end_time - start_time, 2)
converted_time = conver_time(run_time)
print(f"Run time for {NUM_EPOCHS} epoch(s) is {converted_time} seconds.")
print("done")

  0%|          | 0/32 [00:00<?, ?it/s]

Epoch: 0 Loss D.: 50.00044631958008
Epoch: 0 Loss G.: 8.099625587463379



  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

...

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

Run time for 10 epoch(s) is 4.73 seconds.
done


## Visualising Generator and Discriminator Loss

In [ ]:
plt.plot(epoch_count, generator_loss_values, label="Generator loss")
plt.plot(epoch_count, discriminator_loss_values, label="Discriminator loss")
plt.title("Generator and Discriminator loss curves")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.savefig("Test Results/loss_curve.png")
plt.legend();

## Generating synthetic data and save generator model

In [15]:
torch.manual_seed(RANDOM_SEED)
latent_space_samples = torch.randn((1000, 3), device = DEVICE)
generated_samples = generator(latent_space_samples)

generated_samples = generated_samples.detach().numpy()
df = pd.DataFrame(generated_samples, columns = ["Duration", "Size","Idle.Time"])
print(df)
df.to_csv("Test Results/generated_dur_size_idle.time.csv", index = False)

     Duration      Size
0   -0.099430 -0.031040
1   -0.116570  0.035047
2   -0.287762  0.034298
3   -0.164122  0.091254
4   -0.078110 -0.051378
..        ...       ...
995 -0.095357 -0.005502
996 -0.110841 -0.021174
997 -0.089764 -0.039157
998 -0.209945  0.062785
999 -0.249876 -0.099508

[1000 rows x 2 columns]


## Visualise synthetic data

### Scatter graph

In [ ]:
plt.scatter(df["Size"], df["Duration"])
plt.title("Relation betweeen flow duration and flow size in synthetic data")
plt.xlabel("Flow Size (bytes)")
plt.ylabel("Flow Duration (s)")
plt.savefig("Test Results/durationVSsize.png")

### Histograms

In [ ]:
fig, ax = plt.subplots(1, 2)

plt.subplot(1, 2, 1)
plt.hist(df["Duration"])
plt.title("Distribution of synthetic flow duration")
plt.xlabel("Flow duration (s)")
plt.ylabel("Frequency")

plt.subplot(1, 2, 2)
plt.hist(df["Size"])
plt.title("Distribution of synthetic flow size")
plt.xlabel("Flow size (bytes)")
plt.ylabel("Frequency")

fig.tight_layout()
plt.show()
plt.savefig("size_dur_distribution.png")

### Boxplots

In [ ]:
fig, ax = plt.subplots(2, 1)
plt.subplot(2, 1, 1)
plt.boxplot(df["Duration"], vert = False)
plt.title("Distribution of synthetic flow duration")
plt.xlabel("Flow duration (ms)")


plt.subplot(2, 1, 2)
plt.boxplot(df["Size"], vert = False)
plt.title("Distribution of synthetic flow size")
plt.xlabel("Flow size (bytes)")

fig.tight_layout()
plt.show()

plt.savefig("size_dur_distribution.png")